# Running Raphtory from PostgresSQL databases

Download lotr csv file for use later, and import all dependencies

In [ ]:
!curl -o /tmp/lotr.csv https://raw.githubusercontent.com/Raphtory/Data/main/lotr.csv

In [ ]:
import csv
import pandas as pd
from pyraphtory.context import PyRaphtory
from pyraphtory.input import ImmutableString
from pyraphtory.input import GraphBuilder
from pyraphtory.sources import Source
from pyraphtory.graph import Row

ctx = PyRaphtory.local()

## 1) Creating new database and ingesting into Raphtory

Install Psycopg - PostgreSQL database adapter for Python

In [ ]:
pip install psycopg2

Connect to your Postgres database: e.g. psycopg2.connect("dbname=postgres user=bob")

In [ ]:
import psycopg2
conn = psycopg2.connect("dbname= user=")

Create cursor to allow Python code to execute PostgreSQL command in a database session.

In [ ]:
cur = conn.cursor()

Create tables in your postgres database, one for vertices, one for edges.

In [ ]:
cur.execute("CREATE TABLE vertices (id serial PRIMARY KEY, time integer, name varchar, type varchar);")

In [ ]:
cur.execute("CREATE TABLE edges (id serial PRIMARY KEY, time integer, source varchar, target varchar, type varchar);")

Insert vertices into vertices table

In [ ]:
cur.execute("INSERT INTO vertices (time, name, type) VALUES (0, 'Frodo', 'Character'),(1, 'Gandalf', 'Character'),(2, 'Gollum', 'Character');"),

Insert edges in edges table

In [ ]:
cur.execute("INSERT INTO edges (time, source, target, type) VALUES (3, 'Frodo','Gandalf','Character'),(4, 'Gandalf','Gollum','Character'),(5, 'Frodo','Gollum','Character');"),

Make the changes to the database persistent

In [ ]:
conn.commit()

In [ ]:
cur.execute("SELECT * FROM edges;")

Create variable to store list of edges

In [ ]:
edges = cur.fetchall()

Add edges onto graph in Raphtory

In [ ]:
graph = ctx.new_graph("examplegraph")

for row in edges:
    timestamp = int(row[1])
    source_node = row[2]
    target_node = row[3]
    edge_type = row[4]
    
    graph.add_vertex(timestamp, source_node, vertex_type="Character")
    graph.add_vertex(timestamp, target_node, vertex_type="Character") 
    graph.add_edge(timestamp, source_node, target_node, edge_type=edge_type) 

In [ ]:
df = graph.at(5).past().execute(ctx.algorithms.generic.EdgeList()).to_df(['from', 'to'])

In [ ]:
df

## 2) Converting CSV to Postgres database and ingesting into Raphtory

In [ ]:
cur.execute("CREATE TABLE edgesfromcsv (id serial PRIMARY KEY, source varchar, target varchar, time integer);")

In [ ]:
cur.execute("COPY edgesfromcsv(source, target, time) FROM '/tmp/lotr.csv' DELIMITER ',';")

In [ ]:
cur.execute("SELECT * FROM edgesfromcsv;")

In [ ]:
edgesfromcsv = cur.fetchall()

In [ ]:
graph = ctx.new_graph("lotrgraph")
for row in edgesfromcsv:
    timestamp = int(row[3])
    source_node = row[1]
    target_node = row[2]
    
    graph.add_vertex(timestamp, source_node, vertex_type="Character")
    graph.add_vertex(timestamp, target_node, vertex_type="Character")  
    graph.add_edge(timestamp, source_node, target_node, edge_type="Character_Co-occurence") 

In [ ]:
def characters_first_appearance(vertex):
    name = vertex.name()
    event = vertex.earliest_activity()
    earliest_appearance = event.time()
    index = event.index()
    return Row(name,earliest_appearance,index)

df = graph \
    .select(characters_first_appearance) \
    .to_df(["name", "earliest_appearance","index"]) 
df